<a href="https://colab.research.google.com/github/bharathbolla/EVA6/blob/Assignment-3/EVA6_%2B_END_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun May 23 14:44:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from __future__ import print_function

import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
from torch.utils.data import Dataset, TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from random import randint

torch.set_printoptions(linewidth=120)

# Setting up the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print("The device available is: ", device)

The device available is:  cuda:0


## Combined Dataset

In [69]:
def to_categorical(y,num_classes=10):
    """ 1-hot encodes a tensor """
    return (np.eye(num_classes, dtype='uint8')[y])
def cat_array(x): ## This function is to add the second input (random numbers ) as one hot encoded vectors
  return torch.from_numpy(np.array([to_categorical(xi) for xi in x]) )

In [20]:
# Create a class to combine MNIST dataset and random numbers between 0 and 9
class Combined_Dataset():

  # We pass the train variable to get train or test data, and batch_size
  def __init__(self, train, batch_size):

      self.batch_size = batch_size
      # Load the MNIST data into the data_loader object
      self.data_loader = torch.utils.data.DataLoader(
          torchvision.datasets.MNIST('/files/', train=train, download=True,
                                transform=torchvision.transforms.Compose([
                                  torchvision.transforms.ToTensor(),
                                  torchvision.transforms.Normalize(
                                    (0.1307,), (0.3081,))
                                ])),
          batch_size=self.batch_size, shuffle=True)

      # Number of samples in the dataaset
      self.dataset = self.data_loader.dataset            

  # getitem function creats batches of our dataset on the fly by calling next(iter())
  def __getitem__(self, index):
      # Extract one batch of the MNIST data_loader
      image, label = next(iter(self.data_loader))

      # Generate randoms numbers between 0 and 9 of size=batch_size. The datatype is float as this is the input required for the network
      random_numbers = torch.tensor([randint(0,9) for _ in range(self.batch_size)], dtype=torch.int8)
      #print(random_numbers.shape)
      np_random_numbers = random_numbers.numpy()
      #print("NUmpy Random Numbers", np_random_numbers.shape, type(np_random_numbers))
      # Combine inputs and outputs as a list after transfering the tensors to the GPU
      cat_rand_array = cat_array(np_random_numbers)
      #print("Cat Random Numbers Input shape", cat_rand_array.shape)
      x = [image.to(device), cat_rand_array.to(device)]
      #print("Random Numbers Input shape", x[1].shape)
      #x = [image.to(device), random_numbers.to(device)]
      # y labels for addition of number is reshaped to [32,1] as MSE requires it in this format
      #y = [label.to(device), (label+random_numbers).reshape([32,1]).to(device)]
      y = [label.to(device), (label+random_numbers).to(device)]
      #print(y[0],y[1])


      return x, y

  def __len__(self):
      return len(self.data_loader)

# Set the batch size
batch_size = 32

# Create the train and test dataset
train_data = Combined_Dataset(train=True, batch_size=batch_size)
test_data = Combined_Dataset(train=False, batch_size=batch_size)

In [21]:
print(f"Number of train batches: {len(train_data)}")
print(f"Number of test batches: {len(test_data)}")

print(f"Number of train samples: {len(train_data.dataset)}")
print(f"Number of test samples: {len(test_data.dataset)}")

x, y = next(iter(train_data))

print(f"Shape of input data is: [{x[0].shape}, {x[1].shape}]")
print(f"Shape of output data is: [{y[0].shape}, {y[1].shape}]")

Number of train batches: 1875
Number of test batches: 313
Number of train samples: 60000
Number of test samples: 10000
Shape of input data is: [torch.Size([32, 1, 28, 28]), torch.Size([32, 10])]
Shape of output data is: [torch.Size([32]), torch.Size([32])]


## Model

In [103]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv1 = nn.Conv2d(1,32, 3,padding=0,bias=False ) #28
    self.conv2 = nn.Conv2d(32,16, 3,padding=0,bias=False ) #26
    self.conv3 = nn.Conv2d(16, 12,3, padding=0, bias=False)
    self.conv4 = nn.Conv2d(12,10,5, padding=0, bias=False)
    self.linear1 = nn.Linear(10, 100)
    self.linear2 = nn.Linear(100,10)
    self.linear3 = nn.Linear(20,100)
    self.linear4 = nn.Linear(100,19)
    self.flatten = nn.Flatten()

  def forward(self, X, Y):
    X = F.relu(self.conv1(X)) #26
    X = F.relu(self.conv2(X)) #24
    X = F.max_pool2d(X, 2)   #12
    X = F.relu(self.conv3(X)) #10
    X = F.max_pool2d(X, 2) #5x5 16
    X = F.relu(self.conv4(X))   #1x1 10
    X = self.flatten(X) # 10
    X1 = X
    X = F.relu(self.linear1(X))
    X = self.linear2(X)
    Y = Y.type(torch.cuda.FloatTensor)
    Y = torch.cat((X1,Y),1) # This will result in 20 dimensions
    Y = F.relu(self.linear3(Y))
    Y = self.linear4(Y)
    return X, Y

In [7]:
!pip install torchsummary

In [104]:
# Setting up the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print("The device available is: ", device)

The device available is:  cuda:0


In [105]:
# Create an object of the class Network and transfer it to the GPU
model = Model().to(device)
print(" The model layers are: ")
print(model)

print("\nShape of parameters in each layer is: ")
for name, param in model.named_parameters():
    print(name, '\t\t', param.shape)

# Selecting the loss function and optimizer  for the model
CE_loss = nn.CrossEntropyLoss()
MSE_loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

 The model layers are: 
Model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv3): Conv2d(16, 12, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv4): Conv2d(12, 10, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (linear1): Linear(in_features=10, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=10, bias=True)
  (linear3): Linear(in_features=20, out_features=100, bias=True)
  (linear4): Linear(in_features=100, out_features=19, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

Shape of parameters in each layer is: 
conv1.weight 		 torch.Size([32, 1, 3, 3])
conv2.weight 		 torch.Size([16, 32, 3, 3])
conv3.weight 		 torch.Size([12, 16, 3, 3])
conv4.weight 		 torch.Size([10, 12, 5, 5])
linear1.weight 		 torch.Size([100, 10])
linear1.bias 		 torch.Size([100])
linear2.weight 		 torch.Size([10, 100])
linear2.bias 		 torch.Size([10])
linear3.weig

## Training the model

In [106]:
for epoch in range(10):  # Loop over the dataset multiple times

    total_loss = 0.0
    total_correct_1, total_correct_2 = 0, 0
    # Loop over the entire length of train data
    for i in range(len(train_data)):
        # Get the inputs and outputs
        # Input data x is a list of [images, random numbers], output data y is a list of [classes, sum of numbers]
        x, y = next(iter(train_data))

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + Backward + Optimize
        output1, output2 = model(x[0], x[1])
        loss = CE_loss(output1, y[0]) + CE_loss(output2, y[1])
        loss.backward()
        optimizer.step()

        # Calculate statistics
        total_loss += loss.item()
        total_correct_1 += output1.argmax(dim=1).eq(y[0]).sum().item()
        total_correct_2 += output2.argmax(dim=1).eq(y[1]).sum().item()
        
    # Print statistics        
    print(f"Epoch: {epoch+1}, loss: {total_loss}, Classification Acc: {100 * (total_correct_1/(len(train_data.dataset)))}, Addition Acc: {100 * (total_correct_2/(len(train_data.dataset)))}")
   
    
print('Finished Training')

Epoch: 1, loss: 3274.9952426701784, Classification Acc: 90.94333333333333, Addition Acc: 56.49166666666666
Epoch: 2, loss: 493.1207237690687, Classification Acc: 97.605, Addition Acc: 96.64833333333334
Epoch: 3, loss: 332.56705613806844, Classification Acc: 98.015, Addition Acc: 97.41833333333332
Epoch: 4, loss: 262.2368424870074, Classification Acc: 98.34, Addition Acc: 97.89
Epoch: 5, loss: 237.37269268836826, Classification Acc: 98.50999999999999, Addition Acc: 98.095
Epoch: 6, loss: 207.51043169386685, Classification Acc: 98.65833333333333, Addition Acc: 98.29333333333334
Epoch: 7, loss: 193.6566683165729, Classification Acc: 98.71166666666666, Addition Acc: 98.35833333333333
Epoch: 8, loss: 201.59544334141538, Classification Acc: 98.66, Addition Acc: 98.32666666666667
Epoch: 9, loss: 191.7558812100906, Classification Acc: 98.71, Addition Acc: 98.37333333333333
Epoch: 10, loss: 167.6414496623911, Classification Acc: 98.885, Addition Acc: 98.62166666666667
Finished Training


## Evaluating the model

In [107]:
test_correct_1, test_correct_2 = 0, 0
total_1, total_2 = 0, 0

# Since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    # Loop over the entire length of test data
    for i in range(len(test_data)):
        # Get the inputs and outputs
        # Input data x is a list of [images, random numbers], output data y is a list of [classes, sum of numbers]
        x, y = next(iter(test_data))

        # Calculate outputs by running data through the network 
        output1, output2 = model(x[0], x[1])
        
        total_1 += y[0].size(0)
        test_correct_1 += output1.argmax(dim=1).eq(y[0]).sum().item()

        total_2 += y[1].to(device).size(0)
        test_correct_2 += output2.argmax(dim=1).eq(y[1]).sum().item()
        

print('Accuracy of the network on the 10,000 test images: ', (100 * test_correct_1 / total_1))
print('Accuracy of the network on the 10,000 test random number input: ', (100 * test_correct_2 / total_2))

Accuracy of the network on the 10,000 test images:  98.47244408945687
Accuracy of the network on the 10,000 test random number input:  98.18290734824281


## Batch Prediction

In [108]:

print("Print Prediction for a sample batch\n")
x, y = next(iter(test_data))

# Calculate outputs by running data through the network 
output1, output2 = model(x[0], x[1])

# The class with the highest energy is what we choose as prediction
_, predicted = torch.max(output1.data, 1)

print("Predicted vs. Actual for classifer\n")
print(torch.stack((predicted, y[0]), dim=1)[:10])

Print Prediction for a sample batch

Predicted vs. Actual for classifer

tensor([[1, 1],
        [5, 5],
        [2, 2],
        [2, 2],
        [3, 3],
        [6, 6],
        [0, 0],
        [9, 9],
        [9, 9],
        [3, 3]], device='cuda:0')


In [109]:
x[1].argmax(dim=1)

tensor([8, 8, 7, 1, 0, 2, 7, 5, 1, 0, 3, 5, 8, 2, 1, 7, 6, 2, 4, 3, 8, 6, 7, 0, 2, 3, 9, 9, 2, 1, 5, 7],
       device='cuda:0')

In [110]:
y[0]

tensor([1, 5, 2, 2, 3, 6, 0, 9, 9, 3, 2, 2, 1, 7, 7, 2, 1, 9, 5, 0, 6, 1, 3, 1, 8, 4, 1, 3, 0, 5, 7, 9],
       device='cuda:0')

In [111]:
y[1]

tensor([ 9, 13,  9,  3,  3,  8,  7, 14, 10,  3,  5,  7,  9,  9,  8,  9,  7, 11,  9,  3, 14,  7, 10,  1, 10,  7, 10, 12,
         2,  6, 12, 16], device='cuda:0')

In [112]:
## Random Number prediction
output2.argmax(dim=1)

tensor([ 9, 13,  9,  3,  3,  8,  7, 14, 10,  3,  5,  7,  9,  9,  8,  9,  7, 11,  9,  3, 14,  7, 10,  1, 10,  7, 10, 12,
         2,  6, 12, 16], device='cuda:0')